<a href="https://colab.research.google.com/github/Korniev/Data-Sciense/blob/main/hw_11/hw_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Домашнє завдання № 11

В якості домашнього завдання вам пропонується створити рекурентну нейронну мережу за допомогою механізмів Keras, яка буде класифікувати рецензії із датасету imdb.

На відміну від прикладу в модулі 9 ми використаємо рекурентну нейронну мережу. Поекспериментуйте з будовою мережі - RNN, LSTM, двостороння та глибока.

Порівняйте результати та зробіть висновки.

Імпортуємо усі потрібні нам бібліотеки:

In [1]:
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

##Звичайна RNN

Супер, почнемо з реалізаціїї звичайної базової RNN і подивимось на результати, які ми отримаємо.

Для початку потрібно завантажити датасет imdb

Для цього встановлюємо гіперпараметри:

In [2]:
max_features = 10000
maxlen = 500

І завантажимо датасет imdb:

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 [==============================] - 0s 0us/step
